# SPACEc: Using TissUUmaps for interactive visualization
Interactive visualization via TissUUmaps might be informative during multiple steps of the analysis. Apart from the general function provided with the TissUUmaps Python package, we provide specific functions that automatically phrase the input during multiple steps of the analysis.

## Instructions

To use the TissUUmaps viewer you need:
- A pickle file that contains the segmentation output and images 
- An AnnData object containing the currently used single cell data

The *tm_prepare_input* function reads the named content for one region. For that, the user has to provide a region column and a region name. The pickle file has to match the specified region. 
The function creates a folder that contains all necessary input files that are needed to launch the TissUUmaps session. Additionally, the function can launch the TissUUmaps session. If the session is launched from the function a tmap file is created in the input directory that allows to open the session again (both from jupyter and the standalone viewer app).
Alternatively, the function can be used to prepare the directory and the viewer can be launched separately to modify the display options in jupyter as well as host ports etc.

If the Jupyter viewer is too small (might be a problem on small monitors), the user can use the link (displayed if function is executed) to display TissUUmaps in the browser. 

In [3]:
# import spacec first
import spacec as sp

# silencing warnings
import warnings
warnings.filterwarnings('ignore')

#import standard packages
import os
import scanpy as sc


In [7]:
# Specify the path to the data
root_path = "/home/user/path/SPACEc/" # inset your own path

data_path = root_path + 'example_data/raw/' # where the data is stored

# where you want to store the output
output_dir = root_path + 'example_data/output/'

os.makedirs(output_dir, exist_ok=True)

In [8]:
# read in the annotated anndata
adata = sc.read(output_dir + 'adata_nn_demo_annotated_cn.h5ad')
adata

AnnData object with n_obs × n_vars = 46789 × 59
    obs: 'DAPI', 'x', 'y', 'area', 'region_num', 'unique_region', 'condition', 'leiden_1', 'leiden_1_subcluster', 'cell_type_coarse', 'cell_type_coarse_subcluster', 'cell_type_coarse_f', 'cell_type_coarse_f_subcluster', 'cell_type_coarse_f_f', 'cell_type', 'CN_k20_n40', 'CN_k20_n30', 'CN_k20_n20', 'CN_k20_n25', 'CN_k20_n6', 'CN_k20_n6_annot'
    uns: 'CN_k20_n6_colors', 'Centroid_k20_n20', 'Centroid_k20_n25', 'Centroid_k20_n30', 'Centroid_k20_n40', 'Centroid_k20_n6', 'cell_type_coarse_f_colors', 'cell_type_colors', 'dendrogram_cell_type_coarse_f_subcluster', 'leiden', 'leiden_1_colors', 'neighbors', 'ppa_result_100', 'ppa_result_150', 'ppa_result_200', 'ppa_result_250', 'ppa_result_50', 'triDist', 'triDist_keyname', 'umap', 'unique_region_colors'
    obsm: 'X_pca', 'X_umap'
    layers: 'scaled'
    obsp: 'connectivities', 'distances'

In [9]:
adata.obs.head()

,DAPI,x,y,area,region_num,unique_region,condition,leiden_1,leiden_1_subcluster,cell_type_coarse,...,cell_type_coarse_f,cell_type_coarse_f_subcluster,cell_type_coarse_f_f,cell_type,CN_k20_n40,CN_k20_n30,CN_k20_n20,CN_k20_n25,CN_k20_n6,CN_k20_n6_annot
0,105.993197,4.986395,1472.238095,147.0,1,reg002,tonsillitis,23,"23,1",Neutrophil,...,Neutrophil,Neutrophil,Neutrophil,Neutrophil,5,5,5,5,5,Marginal Zone B-DC-Enriched
1,123.677686,5.359504,1322.851240,242.0,1,reg002,tonsillitis,13,13,M2 Macrophage,...,M2 Macrophage,M2 Macrophage,M2 Macrophage,M2 Macrophage,1,1,1,1,1,Marginal Zone
2,107.203125,5.710938,1506.226562,256.0,1,reg002,tonsillitis,23,"23,1",Neutrophil,...,Neutrophil,Neutrophil,Neutrophil,Neutrophil,5,5,5,5,5,Marginal Zone B-DC-Enriched
3,49.660959,8.544521,641.938356,292.0,1,reg002,tonsillitis,21,21,recluster,...,B cell,B cell,B cell,B cell,4,4,4,4,4,Germinal Center
4,148.702532,9.006329,1303.702532,158.0,1,reg002,tonsillitis,12,"12,1",recluster,...,CD4+ T cell,CD4+ T cell,CD4+ T cell,CD4+ T cell,1,1,1,1,1,Marginal Zone


## Integrated use

This function allows the user to reshape the data for TissUUmaps and plot cells from a selected region on top of the original image.

In [11]:
#create cache direction to store tissuumaps cache
os.makedirs(output_dir + "cache", exist_ok=True)

image_list, csv_paths = sp.tl.tm_viewer(
    adata,
    images_pickle_path= output_dir + 'seg_output_tonsil2.pickle',
    directory = output_dir + "cache", # Or inset your own path where you want to cache your images for TM visualization (you can delete this once you are done with TM)
    region_column = "unique_region",
    region = "reg002",
    xSelector = "y",
    ySelector = "x",
    color_by = "cell_type",
    keep_list = None,
    open_viewer=True)

Preparing TissUUmaps input...
Opening TissUUmaps viewer...
Creating project file /home/tim/Dokumente/SPACEc_Apr_2024/cache/reg002/reg002_project.tmap
Loading url:  http://localhost:5100/reg002_project.tmap?path=home/tim/Dokumente/SPACEc_Apr_2024/cache/reg002


## Interactive Catplot via the TissUUmaps viewer

This function starts a simplified version that only shows the cell centroid without the original image. It can be used for fast and interactive visualization. Different from the function above, this function allows visualizing all regions at once. 

In [ ]:
sp.tl.tm_viewer_catplot(
    adata, # anndata object
    directory=None, # directory to save the generated csv files
    region_column="unique_region", # column with the region information (user can select region in tm viewer)
    x="x", # x coordinates
    y="y", # y coordinates
    color_by="cell_type", # cathegorical column to color by
    open_viewer=True, # open the tm viewer 
    add_UMAP=True, # add UMAP to the tm viewer for exploring the feature space along with the spatial data
    keep_list=None) # List of columns to keep from `adata.obs`. If None, defaults to [region_column, x, y, color_by]